# Blender Parameterization Helper Testing


## Imports & Setup



## Load & Parse Code



### Import Test Code


In [211]:

test_code = """
###############
### Imports ###
###############

import bpy
import math


############
### Prep ###
############

# Clear Scene
bpy.ops.object.select_all(action='DESELECT')
bpy.ops.object.select_by_type(type='MESH')
bpy.ops.object.delete()

##################
### Parameters ###
##################

weave_name = "Orbital Vipera Beerus Lace"

large_ring_minor = 0.18
large_ring_major = 1.42

small_ring_minor = 0.17
small_ring_major = 1.00

dist_1 = 1.50
dist_2 = 0.30
dist_3 = 1.20

num_units = 12

angle_1 = math.radians(73)
angle_2 = math.radians(164)
angle_3 = math.radians(90)


##################
### Model Code ###
##################

# Initial Small Ring
bpy.ops.mesh.primitive_torus_add(
    align="WORLD",
    location=(0,0,0),
    rotation=(angle_1,0,0),
    major_radius=small_ring_major,
    minor_radius=small_ring_minor,
    major_segments=48,
    minor_segments=12,
)

for i in range(num_units):
    base_dist = dist_1 * (i+1)
    
    mod = -1 if i % 2 == 0 else 1

    # Vipera Small Ring
    bpy.ops.mesh.primitive_torus_add(
        align="WORLD",
        location=(base_dist,0,0),
        rotation=(mod * angle_1,0,0),
        major_radius=small_ring_major,
        minor_radius=small_ring_minor,
        major_segments=48,
        minor_segments=12,
    )

    # Lace Small Ring
    if i != 0 and i != num_units-1:
        bpy.ops.mesh.primitive_torus_add(
            align="WORLD",
            location=(base_dist,-mod * dist_3,0),
            rotation=(angle_3,0,0),
            major_radius=small_ring_major,
            minor_radius=small_ring_minor,
            major_segments=48,
            minor_segments=12,
        )

    # Large Ring
    bpy.ops.mesh.primitive_torus_add(
        align="WORLD",
        location=(base_dist,mod * dist_2,0),
        rotation=(0,angle_2,0),
        major_radius=large_ring_major,
        minor_radius=large_ring_minor,
        major_segments=48,
        minor_segments=12,
    )

# Final Small Ring
mod = -1 if num_units % 2 == 0 else 1
base_dist = dist_1 * (num_units + 1)
bpy.ops.mesh.primitive_torus_add(
    align="WORLD",
    location=(base_dist,0,0),
    rotation=(mod * angle_1,0,0),
    major_radius=small_ring_major,
    minor_radius=small_ring_minor,
    major_segments=48,
    minor_segments=12,
)
"""



### Split Code By Section


In [212]:

# Set up section containers
section_mapping = {
    "Imports":   "",
    "Parameters": "",
    "Model Code": "",
}

# Initialize section tracking variable
current_section = None

# Itterate over lines to get sections of interest
for line in test_code.split('\n'):

    # Check for section headers
    if ("### " in line) and (" ###" in line):
        current_section = line.replace("#","").strip()

    # Itterate over lines in a target section
    if current_section in section_mapping:

        # Skip useless lines
        if line.strip() == "": continue # empty lines
        if line.startswith("#"): continue # comments

        # Add line to the current section
        section_mapping[current_section] += line + "\n"



### Parse Parameters


In [213]:

parameters = dict()

# Parse parameters section lines for name
for line in section_mapping["Parameters"].split('\n'):
    if "=" in line:

        # Get Values from lines
        split_vals = line.split("=")
        name, value = map(str.strip, split_vals)

        # Check type of value
        value_type = None
        if '"' in  value or "'" in value:
            value_type = "str"
        elif value.replace('.', '', 1).isdigit():
            value_type = "number"
        elif "radians" in value:
            value_type = "angle"
            value = value.replace("radians(", "").replace(")", "").replace("math.", "").strip()
        else:
            value_type = "Unknown"

        parameters[name] = [value, value_type]



## Create Output Code



### Initialize Output Variable


In [214]:

output = """
###############
### Imports ###
###############

"""



### Add Imports to Output


In [215]:

# Check for bpy import
imports_bpy = False
for line in section_mapping["Imports"].split('\n'):
    if line.strip() == "import bpy":
        imports_bpy = True

# Import bpy if not present
if not imports_bpy:
    section_mapping["Imports"] = "import bpy\n" + section_mapping["Imports"]

output += section_mapping["Imports"] + "\n\n"



### Set Weave Name if Provided


In [216]:

weave_name = parameters.get("weave_name")
if weave_name is not None:
    weave_name = weave_name[0]
    output += "################\n### Settings ###\n################\n\n# Weave Name\n"
    output += f"weave_name = {weave_name}\n\n\n"



### Create Update Model Function using Code Section


In [217]:

# Add section header
output += "#############################\n### Model Update Function ###\n#############################\n\n"

# Add function definition
output += "def update_model(self, context):\n\n"

# Add clean up section
output += "\t################\n\t### Clean Up ###\n\t################\n\n"
output += "\t# Clear Scene - Delete all Objects\n"
output += "\tbpy.ops.object.select_all(action='DESELECT')\n"
output += "\tbpy.ops.object.select_by_type(type='MESH')\n"
output += "\tbpy.ops.object.delete()\n\n"

# Add Code  header
output += "\t####################\n\t### Create Rings ###\n\t####################\n\n"

# Replace parameters in model code with self.<parameter_name>
model_code = section_mapping["Model Code"]
for key in parameters:
    model_code = model_code.replace(key, f"self.{key}")

# Clean up cases where "self.self." could appear (if one parameter name is a substring of another e.g. "angle_1" and "angle_10")
while "self.self." in model_code:
    model_code = model_code.replace("self.self.", "self.")

# Prepend tabs and add to output
for line in model_code.split('\n'):
    output += "\t" + line + "\n"



## Display Updated Code


In [218]:

print(output)



###############
### Imports ###
###############

import bpy
import math


################
### Settings ###
################

# Weave Name
weave_name = "Orbital Vipera Beerus Lace"


#############################
### Model Update Function ###
#############################

def update_model(self, context):

	################
	### Clean Up ###
	################

	# Clear Scene - Delete all Objects
	bpy.ops.object.select_all(action='DESELECT')
	bpy.ops.object.select_by_type(type='MESH')
	bpy.ops.object.delete()

	####################
	### Create Rings ###
	####################

	bpy.ops.mesh.primitive_torus_add(
	    align="WORLD",
	    location=(0,0,0),
	    rotation=(self.angle_1,0,0),
	    major_radius=self.small_ring_major,
	    minor_radius=self.small_ring_minor,
	    major_segments=48,
	    minor_segments=12,
	)
	for i in range(self.num_units):
	    base_dist = self.dist_1 * (i+1)
	    mod = -1 if i % 2 == 0 else 1
	    # Vipera Small Ring
	    bpy.ops.mesh.primitive_torus_add(
	  